In [3]:
import os
import sys

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir, os.pardir)))


import DifferentiationInterface.src.diffipy.diffipy as dp

In [4]:
backend_array = ['numpy', 'torch',  'tensorflow', 'jax', 'aadc'] # aadc not pre-computing tape yet...

for backend in backend_array:
    dp.setBackend(backend)
    # Initialize variables and constants
    s0_value = 100.0
    K_value = 110.0
    r_value = 0.05
    sigma_value = 0.3
    dt_value = 1.0
    z_value = 0.5
    N = 1000000
    seed = 1

    # Define variables
    s0 = dp.variable(s0_value, 'input','s0')
    K = dp.variable(K_value, 'input','K')
    r = dp.variable(r_value, 'input','r')
    sigma = dp.variable(sigma_value, 'input','sigma')
    dt = dp.variable(dt_value, 'input','dt')
    z = dp.variable(z_value, 'randomVariableNormal','z')

    func_input_variables = [s0, K, r, sigma, dt, z]
    diff_variables = [s0, K, r, sigma, dt]

    # Record Tape: Standard Monte Carlo

    s = s0 * dp.exp((r - sigma **2 / 2) * dt + sigma * dp.sqrt(dt) * z)
    payoff =  dp.if_(s > K, s - K, 0)
    PV_standard = dp.exp(-r * dt) * dp.sum(payoff) / N

    ### Test performance # seed and NewSample not working for aadc yet
    #dp.seed(seed)
    #pre_computed_random_variables = z.NewSample(N) # pre-evaluate random samples

    PV_standard.run_performance_test(func_input_variables, diff_variables, warmup_iterations = 3, test_iterations = 5)


Backend              Eval-Result  mean runtime    variance runtime    gradient directions: {'K', 's0', 'r', 'dt', 'sigma'}
numpy                10.041970    0.077220        0.000000            [0.5001524653991396, -0.36339342042168704, 39.97366803307756, 39.9872181960248, 7.9967309575934555]
numpy_as_func        10.041970    0.072629        0.000001            {'s0': 0.5001524975511984, 'K': -0.36339337601276606, 'r': 39.97366806132163, 'sigma': 39.98721820632767, 'dt': 7.996730952442021}
torch                10.035641    0.014930        0.000127            [0.4997710883617401, -0.36310428380966187, 39.94146728515625, 39.96198654174805, 7.9913716316223145]
torch_as_func        10.035641    0.001173        0.000000            [tensor(0.4998), tensor(-0.3631), tensor(39.9415), tensor(39.9620), tensor(7.9914)]
tensorflow           10.029771    0.031398        0.000034            [0.49989134, -0.36326697, 39.959366, 39.94162, 7.989212]
tensorflow_as_func   10.029771    0.009007        0.00

TypeError: __init__(): incompatible constructor arguments. The following argument types are supported:
    1. aadc._aadc_core.idouble(arg0: float)
    2. aadc._aadc_core.idouble(arg0: aadc._aadc_core.idouble)

Invoked with: array([-0.57897872,  1.92670371, -1.25380562, ...,  0.88467465,
        0.86908353, -0.04137146])